In [100]:
import pandas as pd
import requests

# --- CONFIGURATION ---
SCHOOL_QUALITY_BASE_URL = "https://data.cityofnewyork.us/resource/dnpx-dfnc.json"
SCHOOL_DEMOGRAPHICS_BASE_URL = "https://data.cityofnewyork.us/resource/c7ru-d68s.json"
APP_TOKEN = None  # Optional: Add your Socrata app token if you have one

In [101]:
# --- GENERIC QUERY BUILDER ---
def build_soql_query(select_fields=None, where_clause=None, limit=5000, offset=0, order_by=None):
    params = {}
    if select_fields:
        params['$select'] = ','.join(select_fields)
    if where_clause:
        params['$where'] = where_clause
    if limit:
        params['$limit'] = limit
    if offset:
        params['$offset'] = offset
    if order_by:
        params['$order'] = order_by
    return params

# --- DATA FETCHER ---
def fetch_data(base_url, params):
    headers = {}
    if APP_TOKEN:
        headers['X-App-Token'] = APP_TOKEN

    response = requests.get(base_url, params=params, headers=headers)
    response.raise_for_status()
    return pd.DataFrame(response.json())

# --- JOINER ---
def join_quality_and_demographics(quality_df, demographics_df):
    # Normalize DBN fields if necessary
    return pd.merge(quality_df, demographics_df, left_on='dbn', right_on='dbn', how='left')

In [102]:
quality_select_fields = [
    "dbn", "school_name", "report_type", "metric_display_name", "metric_variable_name", "metric_value",
    "metric_score", "number_of_students", "school_year", "report_year", "school_type"
]

quality_params = build_soql_query(
    select_fields=quality_select_fields,
    where_clause="report_year = 2024 AND metric_value IS NOT NULL",
    limit=500000
)

quality_df = fetch_data(SCHOOL_QUALITY_BASE_URL, quality_params)
quality_df['metric_value'] = pd.to_numeric(quality_df['metric_value'], errors='coerce')
print(f"Fetched {len(quality_df)} rows of school quality data.")


Fetched 166141 rows of school quality data.


In [104]:
quality_df['metric_display_name'].unique()


array(['Average Student Attendance',
       'Percentage of Students with >90% Attendance',
       'Percentage of Students with >90% Attendance - Black',
       'Percentage of Students with >90% Attendance - Hispanic',
       'Percentage of Students with >90% Attendance - Female',
       'Percentage of Students with >90% Attendance - Male',
       'MS Adjusted Core Course Pass Rate of Former Students',
       'Movement of Students with IEPs to Less Restrictive Environments',
       'ELL Progress',
       'Percentage of Students at Level 3 or 4, ELA, Grade 3',
       'Percentage of Students at Level 3 or 4, ELA, Grade 4',
       'Percentage of Students at Level 3 or 4, ELA, Grade 5',
       'Percentage of Students at Level 3 or 4, ELA',
       'Percentage of Students at Level 3 or 4, ELA - Black',
       'Percentage of Students at Level 3 or 4, ELA - Hispanic or Latinx',
       'Percentage of Students at Level 3 or 4, ELA - Female',
       'Percentage of Students at Level 3 or 4, ELA - M

In [112]:
# Define filter conditions as a list of tuples: (metric_variable_name, min_value)
conditions = [
    ('attendance_k8_all', 0.90),
    ('chronic_absent_ems_all', 0.85),
    ('rating_mean_ela_all', 2.5),
    ('rating_mean_mth_all', 2.5),
    ('prof_pct_ela_3gr', 0.40),
    ('prof_pct_mth_3gr', 0.40)
]

# Create a mask: start True for all schools
schools_passing = pd.Series(True, index=quality_df['dbn'].unique())

for metric, threshold in conditions:
    # Find rows matching the metric
    metric_df = quality_df[quality_df['metric_variable_name'] == metric]

    # Schools that meet the threshold for this metric
    passing_this_metric = metric_df[metric_df['metric_value'] >= threshold]['dbn'].unique()

    # Update mask: only keep schools passing all conditions
    schools_passing &= schools_passing.index.isin(passing_this_metric)

# Filter final schools
final_dbns = schools_passing[schools_passing].index

# (Optional) Join back to school names or more info
final_schools = quality_df[quality_df['dbn'].isin(final_dbns)][['dbn', 'school_name']].drop_duplicates()

print(final_schools)

# Save results
final_schools.to_csv('filtered_schools_compound_tidy.csv', index=False)


           dbn                                        school_name
130     01M184                               P.S. 184m Shuang Wen
1697    01M539  New Explorations into Science, Technology and ...
1749    25Q130                                           P.S. 130
2454    02M006                           P.S. 006 Lillie D. Blake
2740    02M041                         P.S. 041 Greenwich Village
...        ...                                                ...
159206  84X255                Bronx Charter School for Excellence
162874  84X538                             Icahn Charter School 5
163261  84X579              Bronx Charter School for Excellence 2
163739  84X589              Bronx Charter School for Excellence 3
164216  84X608              Bronx Charter School for Excellence 4

[78 rows x 2 columns]


In [106]:
demographics_select_fields = [
    "dbn", "school_name", "year",
    "total_enrollment", "english_language_learners_1", "students_with_disabilities_1",
    "asian_1", "black_1", "hispanic_1", "white_1", "missing_race_ethnicity_data_1",
    "poverty_1", "economic_need_index"
]

demographics_params = build_soql_query(
    select_fields=demographics_select_fields,
    where_clause="year='2021-22'",
    limit=5000
)

demographics_df = fetch_data(SCHOOL_DEMOGRAPHICS_BASE_URL, demographics_params)
print(f"Fetched {len(demographics_df)} rows of school demographics data.")


Fetched 1881 rows of school demographics data.


In [ ]:
combined_df = join_quality_and_demographics(quality_df, demographics_df)
print(f"Joined dataset: {combined_df.shape[0]} rows, {combined_df.shape[1]} columns")


In [ ]:
# Convert important columns to numeric
for col in ['metric_score', 'total_enrollment', 'poverty_1',
            'english_language_learners_1', 'students_with_disabilities_1',
            'asian_1', 'black_1', 'hispanic_1', 'white_1']:
    combined_df[col] = pd.to_numeric(combined_df[col], errors='coerce')
    

In [ ]:
# Example: Search for aligned elementary schools
aligned_schools = combined_df[
    (combined_df['school_type'] == 'Elementary') &
    (combined_df['metric_score'] >= 3.5) &
    (combined_df['total_enrollment'] >= 300) &
    (combined_df['poverty_1'] <= 60) &
    (combined_df['white_1'] >= 20) &  # customize based on your own alignment needs
    (combined_df['black_1'] <= 50)
]

# Sort strongest signals first
aligned_schools = aligned_schools.sort_values('metric_score', ascending=False)

# Preview
aligned_schools[['school_name_x', 'dbn', 'metric_score', 'poverty_1',
                 'asian_1', 'black_1', 'hispanic_1', 'white_1']].head(20)


In [108]:
combined_df.head()


,dbn,school_name_x,metric_display_name,metric_value,metric_score,school_year,report_year,school_type,school_name_y,year,total_enrollment,english_language_learners_1,students_with_disabilities_1,asian_1,black_1,hispanic_1,white_1,missing_race_ethnicity_data_1,poverty_1,economic_need_index
0,02M933,City Knoll Middle School,ELL Progress,0.78,3.35,2022,2023,Middle,City Knoll Middle School,2021-22,116,0.155,0.371,0.060,0.207,0.612,0.112,0.0,NaN,80.3%
1,12X211,P.S. 211,Average Student Attendance,0.916,NaN,2022,2023,Elementary,P.S. 211,2021-22,408,0.140,0.201,0.010,0.164,0.804,0.012,0.0,NaN,93.5%
2,11X299,Astor Collegiate Academy,ELL Progress,0.513,NaN,2022,2023,High School,Astor Collegiate Academy,2021-22,450,0.120,0.216,0.091,0.267,0.462,0.151,0.007,NaN,77.9%
3,01M020,P.S. 020 Anna Silver,"Average Student Proficiency, Math, Grade 5",2.21,NaN,2022,2023,Elementary,P.S. 020 Anna Silver,2021-22,351,0.100,0.211,0.197,0.151,0.547,0.077,0.003,NaN,75.3%
4,01M020,P.S. 020 Anna Silver,"Average Student Proficiency, Math, Grade 4",2.27,NaN,2022,2023,Elementary,P.S. 020 Anna Silver,2021-22,351,0.100,0.211,0.197,0.151,0.547,0.077,0.003,NaN,75.3%
